# Extraction des données, nettoyage et normalisation des valeurs et implantation dans un data warehouse

In [2]:
import pandas as pd
import sqlite3

import ETL

La première partie consiste principalement à la création du dataframe général avec la récupération des données depuis les fichiers CSV ainsi que le nettoyage en utilisant notre module créé par nos soins "ETL". 

In [3]:
poleemploi_df, makesense_df, empterritorial_df = ETL.load_sources()

# Normalisation de la date
poleemploi_df = ETL.normalisation_date(poleemploi_df)
empterritorial_df = ETL.normalisation_date(empterritorial_df)
makesense_df = ETL.normalisation_date(makesense_df)

# Normalisation du salaire et du niveau d'expérience
poleemploi_df['salaire'] = [element.lower() for element in poleemploi_df['salaire']]
poleemploi_df['experience'] = [element.lower() for element in poleemploi_df['experience']]

poleemploi_df['salaire_moyen'] = poleemploi_df["salaire"].apply(ETL.normalisation_salaire)
makesense_df['salaire_moyen'] = None
empterritorial_df['salaire_moyen'] = None

poleemploi_df['debutant_acceptee'] = poleemploi_df["experience"].apply(lambda x: "Débutant" if "débutant" in x else "Experience Exigée")
makesense_df['debutant_acceptee'] = "Non spécifié"
empterritorial_df['debutant_acceptee'] = "Non spécifié"

poleemploi_df['annee_exper'] = poleemploi_df["experience"].apply(ETL.normalisation_experience)
makesense_df['annee_exper'] = None
empterritorial_df['annee_exper'] = None

# Normalisation du titre de l'offre (métier recherché)
poleemploi_df["titre"] = poleemploi_df["titre"].apply(ETL.normalisation_metier)
makesense_df["titre"] = makesense_df["titre"].apply(ETL.normalisation_metier)
empterritorial_df["titre"] = empterritorial_df["titre"].apply(ETL.normalisation_metier)

# Normalisation de la localisation de l'offre
poleemploi_df = ETL.normalisation_adresse(poleemploi_df, "geo.csv")
makesense_df = ETL.normalisation_adresse(makesense_df, "geo.csv")
empterritorial_df = ETL.ajout_region(empterritorial_df, "geo.csv")
empterritorial_df["pays"] = "France"

# Normalisation des textes (retrait des emojis propre à Jobs that make sense)
makesense_df["descriptif"] = makesense_df["descriptif"].apply(ETL.normalisation_emojis)
makesense_df["profil"] = makesense_df["profil"].apply(ETL.normalisation_emojis)

# Application de "Non spécifié" quand absence texte
poleemploi_df["profil"] = poleemploi_df["profil"].apply(ETL.normalisation_null)

# Normalisation type contrat
poleemploi_df = ETL.normalisation_typecontrat(poleemploi_df)
makesense_df = ETL.normalisation_typecontrat(makesense_df)

final_poleemploi_df = poleemploi_df[["titre", "descriptif", "profil","nom_departement", "nom_region", "pays", "jour", "mois", "année", "date_complete", 
                                     "source", "type_contrat", "entreprise", "annee_exper", "debutant_acceptee",
                                     "salaire_moyen"]]
final_makesense_df = makesense_df[["titre", "descriptif", "profil","nom_departement", "nom_region", "pays", "jour", "mois", "année", "date_complete", 
                                     "source", "type_contrat", "entreprise", "annee_exper", "debutant_acceptee",
                                     "salaire_moyen"]]
final_empterrotrial_df = empterritorial_df[["titre", "descriptif", "profil","nom_departement", "nom_region", "pays", "jour", "mois", "année", "date_complete", 
                                     "source", "type_contrat", "entreprise", "annee_exper", "debutant_acceptee",
                                     "salaire_moyen"]]

# Création dataframe final
final_df = pd.concat([final_poleemploi_df, final_makesense_df, final_empterrotrial_df])
final_df.reset_index(drop=True, inplace=True)

In [4]:
final_df.shape

(3310, 16)

In [5]:
final_df.head()

,titre,descriptif,profil,nom_departement,nom_region,pays,jour,mois,année,date_complete,source,type_contrat,entreprise,annee_exper,debutant_acceptee,salaire_moyen
0,Data manager,Description du poste : En bref : Nice CDI Data...,Non spécifié,Alpes-Maritimes,Provence-Alpes-Côte d'Azur,France,4.0,1.0,2024.0,2024-01-04,METEOJOB,CDI,Non spécifié,1.0,Experience Exigée,NaN
1,Data manager,Description du poste : En bref : Cagnes sur Me...,Non spécifié,Alpes-Maritimes,Provence-Alpes-Côte d'Azur,France,4.0,1.0,2024.0,2024-01-04,METEOJOB,CDI,Non spécifié,1.0,Experience Exigée,NaN
2,Data manager,Description du poste : LE BRIEF CDI Data Engin...,Non spécifié,Loire-Atlantique,Pays de la Loire,France,4.0,1.0,2024.0,2024-01-04,METEOJOB,CDI,Non spécifié,3.0,Experience Exigée,NaN
3,Data analyst,Description du poste : Responsabilités princip...,Non spécifié,Loire-Atlantique,Pays de la Loire,France,4.0,1.0,2024.0,2024-01-04,METEOJOB,Intérim,Non spécifié,1.0,Experience Exigée,42500.0
4,Data manager,Description du poste : Le saviez vous ? Nous r...,Vous êtes diplômé en informatique au moins au...,Essonne,Île-de-France,France,4.0,1.0,2024.0,2024-01-04,METEOJOB,CDI,Non spécifié,3.0,Experience Exigée,NaN


Pour la seconde partie, nous créons les différentes tables de dimensions et de fais pour ensuite les charger dans notre base de données SQLite.

In [6]:
# Création des différentes tables de dimensions
d_geo = final_df[["nom_departement", "nom_region", "pays"]]
d_geo = d_geo.drop_duplicates()
d_geo["id_geo"] = range(len(d_geo)) # Création clé

d_temps = final_df[["jour", "mois", "année", "date_complete"]]
d_temps = d_temps.drop_duplicates()
d_temps["id_temps"] = range(len(d_temps))

d_source = final_df[["source"]]
d_source = d_source.drop_duplicates()
d_source["id_source"] = range(len(d_source))

d_contrat = final_df[["type_contrat"]]
d_contrat = d_contrat.drop_duplicates()
d_contrat["id_contrat"] = range(len(d_contrat))

d_entreprise = final_df[["entreprise"]]
d_entreprise = d_entreprise.drop_duplicates()
d_entreprise["id_entreprise"] = range(len(d_entreprise))

d_experience = final_df[["annee_exper", "debutant_acceptee"]]
d_experience = d_experience.drop_duplicates()
d_experience["id_experience"] = range(len(d_experience))

d_salaire = final_df[["salaire_moyen"]]
d_salaire = d_salaire.drop_duplicates()
d_salaire["id_salaire"] = range(len(d_salaire))

d_offres = final_df[['titre', 'descriptif', 'profil']]
d_offres = d_offres.drop_duplicates()
d_offres["id_offres"] = range(len(d_offres))

# Création table de faits (en récupérant les ids de toutes les tables)
merge = final_df.merge(d_geo, on=["nom_departement", "nom_region", "pays"])
merge = merge.merge(d_temps, on=["jour", "mois", "année", "date_complete"])
merge = merge.merge(d_source, on=["source"])
merge = merge.merge(d_entreprise, on=["entreprise"])
merge = merge.merge(d_contrat, on=["type_contrat"])
merge = merge.merge(d_experience, on=["annee_exper", "debutant_acceptee"])
merge = merge.merge(d_salaire, on=["salaire_moyen"])
merge = merge.merge(d_offres, on=['titre', 'descriptif', 'profil'])

f_offres = merge[['id_offres',
       'id_geo', 'id_temps', 'id_source', 'id_entreprise', 'id_contrat',
       'id_experience', 'id_salaire']]

# Création DW avec sqllite3
con = sqlite3.connect("joboffers_dw.db")
cur = con.cursor() # Création d'un curseur pour intéragir avec la base de données

# On Drop la table avant de la recréer pour éviter les doublons
cur.execute('DROP TABLE f_offres')
# On crée la table de faits avec les différentes clés étrangères
cur.execute('''
    CREATE TABLE IF NOT EXISTS f_offres (
        id_offres INTEGER,
        id_geo INTEGER,
        id_temps INTEGER,
        id_source INTEGER,
        id_entreprise INTEGER,
        id_contrat INTEGER,
        id_experience INTEGER,
        id_salaire INTEGER,
        FOREIGN KEY (id_offres) REFERENCES d_offres(id_offres)
        FOREIGN KEY (id_geo) REFERENCES d_geo(id_geo),
        FOREIGN KEY (id_temps) REFERENCES d_temps(id_temps),
        FOREIGN KEY (id_source) REFERENCES d_source(id_source),
        FOREIGN KEY (id_entreprise) REFERENCES d_entreprise(id_entreprise),
        FOREIGN KEY (id_contrat) REFERENCES d_contrat(id_contrat),
        FOREIGN KEY (id_experience) REFERENCES d_experience(id_experience),
        FOREIGN KEY (id_salaire) REFERENCES d_salaire(id_salaire)
    )
''')
# On utilise la commande pandas .to_sql() pour envoyer le dataframe dans SQLite
f_offres.to_sql('f_offres', con, if_exists='append',index=False)

cur.execute('DROP TABLE d_offres')
cur.execute('''
    CREATE TABLE IF NOT EXISTS d_offres (
        titre TEXT,
        descriptif TEXT, 
        profil TEXT,
        id_offres INTEGER PRIMARY KEY
        
    )
''')

d_offres.to_sql('d_offres', con,if_exists='append', index=False)

cur.execute('DROP TABLE d_contrat')
cur.execute('''
    CREATE TABLE IF NOT EXISTS d_contrat (
        type_contrat TEXT,
        id_contrat INTEGER PRIMARY KEY
        
    )
''')
d_contrat.to_sql('d_contrat', con, if_exists='append',index=False)

cur.execute('DROP TABLE d_source')
cur.execute('''
    CREATE TABLE IF NOT EXISTS d_source (
        source TEXT,
        id_source INTEGER PRIMARY KEY
        
    )
''')
d_source.to_sql('d_source', con, if_exists='append',index=False)

cur.execute('DROP TABLE d_salaire')
cur.execute('''
    CREATE TABLE IF NOT EXISTS d_salaire (
        salaire_moyen INTEGER,
        id_salaire INTEGER PRIMARY KEY
        
    )
''')
d_salaire.to_sql('d_salaire', con, if_exists='append', index=False)

cur.execute('DROP TABLE d_entreprise')
cur.execute('''
    CREATE TABLE IF NOT EXISTS d_entreprise (
        entreprise TEXT,
        id_entreprise INTEGER PRIMARY KEY
        
    )
''')

d_entreprise.to_sql('d_entreprise', con, if_exists='append', index=False)

cur.execute('DROP TABLE d_experience')
cur.execute('''
    CREATE TABLE IF NOT EXISTS d_experience (
        annee_exper INTEGER,
        debutant_acceptee TEXT,
        id_experience INTEGER PRIMARY KEY
        
    )
''')
d_experience.to_sql('d_experience', con, if_exists='append', index=False)

cur.execute('DROP TABLE d_geo')
cur.execute('''
    CREATE TABLE IF NOT EXISTS d_geo (
        nom_departement TEXT,
        nom_region TEXT,
        pays TEXT,
        id_geo INTEGER PRIMARY KEY
        
    )
''')
d_geo.to_sql('d_geo', con, if_exists='append', index=False)

cur.execute('DROP TABLE d_temps')
cur.execute('''
    CREATE TABLE IF NOT EXISTS d_temps (
        jour INTEGER,
        mois INTEGER,
        année INTEGER,
        date_complete DATETIME,
        id_temps INTEGER PRIMARY KEY
        
    )
''')
d_temps.to_sql('d_temps', con, if_exists='append', index=False)

# On ferme la connection avec la base de données.
con.commit()
con.close()

In [8]:
# Exemple d'une table de dimensions
d_geo.head()

,nom_departement,nom_region,pays,id_geo
0,Alpes-Maritimes,Provence-Alpes-Côte d'Azur,France,0
2,Loire-Atlantique,Pays de la Loire,France,1
4,Essonne,Île-de-France,France,2
7,Hauts-de-Seine,Île-de-France,France,3
8,Somme,Hauts-de-France,France,4
